# NeRF-WandB-COLAB

### [NeRF 공식 저장소](https://github.com/bmild/nerf)를 수정하여 [wandb 연동이 편리하도록 만든 저장소](https://github.com/ProtossDragoon/nerf-wandb.git)의 소스코드를 이용해 커스텀 데이터셋에 NeRF 모델 학습

Plank-ing Hyundong 3D Reconstruction Project
Created 2022.07.12 <br>

**NoteBook Author** <br>
[Janghoo Lee](https://www.linkedin.com/in/janghoo-lee-25212a1a0) <br>
🎓 : [ProtossDragoon](https://github.com/ProtossDragoon) <br>
📧 : dlwkdgn3@gmail.com <br>

🤔 If you have any questions, please raise an issue in our [github repo](https://github.com/ProtossDragoon/PlankHyundong).

## 환경

### GPU 확인

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

### TF 런타임 변경

In [ ]:
try:
    %tensorflow_version 1.x
except ValueError:
    # 만약 %tensorflow_version 1.x magic 명령어가 작동하지 않는 경우
    !pip uninstall --yes tensorflow
    !pip install tensorflow==1.15
    import tensorflow
    print(tensorflow.__version__)

### 의존 패키지 설치

In [ ]:
!sudo apt -qq install imagemagick
!pip install ConfigArgParse -qqq
!pip install imageio-ffmpeg -qqq

### NeRF 소스코드 다운로드

In [ ]:
!git clone https://github.com/ProtossDragoon/nerf-wandb.git

In [ ]:
%cd nerf-wandb
#!git checkout dev # 개발 중
!ls -al

### Weight and Bias 설치

- 실험 관리 도구로 wandb 를 사용합니다.
- 개인 wandb 프로젝트가 아닌, [plank-hyundong](https://wandb.ai/plank-hyundong) 팀의 [wandb 프로젝트 plank-hyundong](https://wandb.ai/plank-hyundong/plank-hyundong) 저장소를 사용하려면 관리자(dlwkdgn1@naver.com)에게 접근 키를 문의하세요.
- 개인 저장소를 사용하려면 `wandb.login()` 셀을 실행할 때 개인 계정으로 로그인하고, `python run_nerf.py` 셀을 실행할 때 파라미터 중 `--wandbproject` 와 `--wandbentity` 값을 변경하면 됩니다. 

In [ ]:
!pip install wandb -qqq

In [ ]:
# Log in to your W&B account    
import wandb
wandb.login()

## 커스텀 데이터로 실행

- LLFF 를 통해 얻은 pose 가 필요합니다.
- LLFF 을 실행하려면 [Plankhyundong 저장소](https://github.com/ProtossDragoon/PlankHyundong)의 [colmap_colab 노트북](https://github.com/ProtossDragoon/PlankHyundong/blob/main/notebooks/colmap_colab.ipynb)을 참고하세요.

### 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

### 파라미터 설정

In [ ]:
from datetime import datetime
now = datetime.now().strftime('%y%m%d_%H%M%S')

dataset_name = 'hyundong360_100' #@param ['hyundong360_50', 'hyundong360_100', 'hyundong360_200', 'hyundong360_400', 'noparking360_100', 'noparking360_200', 'noparking360_400', 'noparking720_100', 'noparking720_200', 'noparking720_400', 'hyundong360_100_cleanup', 'hyundong360_100_background']
downsample_factor = 4 #@param {type:"slider", min:1, max:64, step:1}
netdepth = 6 #@param {type:"slider", min:4, max:16, step:2}
netwidth = 96 #@param {type:"slider", min:64, max:256, step:4}
experiment_name = f'{dataset_name}_{downsample_factor}_downsampled_{now}'
max_iter = 30000 #@param
learning_rate = 0.001 #@param
video_saving_cnt = 3 #@param {type:"slider", min:1, max:10, step:1}
n_samples = 128 #@param {type:"slider", min:32, max:256, step:32}

# fine 모델에서 사용되는 샘플 개수는 coarse 모델의 sampling 개수의 2배로 설정한다.
# 공식 논문에서 제안하는 대로, 64이면 128.
n_importance = n_samples * 2

# Reproduce 를 위해 고정 random_seed 를 사용
random_seed = 777 #@param

# tradeoff: memory <-> speed (training 에는 속도와 성능 모두에 영향을 미치지 않음. 학습 도중 동영상을 만들 때 OOM 이 난다면 충분히 낮출 것)
rendering_speed = 2048 #@param {type:"slider", min:1024, max:16384, step:1024}

# tradeoff: memory <-> result
n_points_per_ray = 65536 #@param {type:"slider", min:2048, max:262144, step:1024}

_dummy_dir = f'./logs/{experiment_name}'
_tensorboard_logdir = f'./logs/summaries/{experiment_name}'
print(f'experiment: {experiment_name}')

### 학습 시작

- 360도에 걸쳐 촬영한 데이터의 경우, `--no_ndc`, `--spherify`, `--lindisp` 를 추가하세요.

In [ ]:
!python run_nerf.py \
    --wandbproject {'plank-hyundong'} \
    --wandbentity {'plank-hyundong'} \
    --maxiter {max_iter} \
    --datadir /content/drive/MyDrive/dev/llff_data/{dataset_name} \
    --dataset_type llff \
    --factor {downsample_factor} \
    --netdepth {netdepth} \
    --netwidth {netwidth} \
    --netdepth_fine {netdepth} \
    --netwidth_fine {netwidth} \
    --chunk {rendering_speed} \
    --netchunk {n_points_per_ray} \
    --lrate {learning_rate} \
    --i_video {max_iter // video_saving_cnt} \
    --expname {experiment_name} \
    --N_samples {n_samples} \
    --N_importance {n_importance} \
    --random_seed {random_seed} \
    --raw_noise_std 1.0 \
    --use_viewdirs \
    --no_ndc \
    --spherify \
    --lindisp \